# 간단한 챗봇 만들기
- 데이터를 기반으로 응답하는 간단한 챗봇입니다.

In [36]:
import os
from http.client import responses

from pyexpat.errors import messages


def load_api_keys(filepath):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

filepath = '../.env'

# API 키 로드 및 환경변수 설정
load_api_keys(filepath)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_list = [
    'MLP-KTLim/llama-3-Korean-Bllossom-8B', # 너무 오래걸림
    'microsoft/Phi-4-mini-instruct',
    'UNIVA-Bllossom/DeepSeek-llama3.1-Bllossom-8B',
]

model = AutoModelForCausalLM.from_pretrained(
    model_list[0],
    cache_dir='../_download/models',
    revision='main',
    # device_map='auto',
    torch_dtype=torch.float16,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_list[0],
    cache_dir='../_download/models',
    revision='main',
    # device_map='auto',
    torch_dtype=torch.float16,
    trust_remote_code=True,
)

D:\workspace\repository\Lab\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\workspace\repository\Lab\_download\models\models--MLP-KTLim--llama-3-Korean-Bllossom-8B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]Xet Storage is enabled for this repo,

In [56]:
from langchain_huggingface import HuggingFacePipeline

generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.1,
    # device=0,
)

llm = HuggingFacePipeline(pipeline=generator)

Device set to use cuda:0


In [61]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

words = ["호랑이", "나무"]

class Result(BaseModel):
    score: float = Field(..., description="연관성 점수, -1.0~1.0")
    reason: str = Field(..., description="점수에 대한 이유")

parser = PydanticOutputParser(pydantic_object=Result)
prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 단어들의 연관성을 판단하는 언어학자야.
    다음 주어지는 단어 두 개의 연관성을 판단하여 점수와 해당 점수를 매긴 이유를 출력해야해.

    다음은 매겨야 할 점수의 상세 설명이야.
    1. 점수의 범위: -1.000 ~ 1.000
    2. 연관성에 따른 점수 -1에 가까울 수록 정반대의 연관성을 가지고 있으며 연관성이 없을 경우 0, 연관성이 많을경우 1에 가까운 점수를 나타냄.

    응답 예시는 다음과 같아.
    {{
        score: 0.002,
        season: 단어1과 단어2는 연관성이 거의 없다고 판단됩니다.
    }}
    답변은 무조건 한국어로 답해야 하며 반드시 응답포맷을 지킬 것.
    """),
    ("human", f"단어1: {words[0]}, 단어2: {words[1]}"),
    ("system", "{format_instructions}")
])

messages = prompt.format_messages(format_instructions=parser.get_format_instructions())
llm = HuggingFacePipeline(pipeline=generator)
response = llm.invoke(messages)

import re

match = re.findall(r'\{\s*"score"\s*:\s*-?\d+\.\d+,\s*"reason"\s*:\s*".*?"\s*\}', response, re.DOTALL)
result: BaseModel = None

if match:
    json_str = match[-1]
    try:
        result = parser.parse(json_str)
    except Exception as e:
        print("파싱 실패:", e)
else:
    print("JSON 블록을 찾을 수 없습니다.")
    display(response)



In [62]:
result

Result(score=-0.7, reason='단어1과 단어2는 서로 다른 주제에 속하며, 호랑이는 동물이고 나무는 식물로, 이 두 가지는 직접적인 연관성이 없습니다.')